In [1]:
#imports
import pandas as pd
import numpy as np
import mdptoolbox
import random as rand
import time
from scipy.sparse import csr_matrix

In [2]:
#opening files
train = pd.read_csv("training.txt", sep='\t', header=None, names=["action", "color"])
#test
test=open("order.txt","r")

In [3]:
#strategic parameters for population of transition matrix
#actionDist="uniform"
actionDist="fromTraining"

fieldSelectBias="uniform"
#fieldSelectBias="closenessBias"

#parameters for environment
length=3
width=2
numcolors=3

#resulting parameters
fieldstates=numcolors+1 #+1 for empty
fields=length*width
numAction=2 *numcolors #store/take for each color

#Hamming distance for each field from origin
distance=[]
for l in range(length):
    for w in range(width):
        distance.append(1+l+w)

#define one warehouse state for every combination of field occupation for each field and every executed action
numstates=fieldstates**fields *numAction

print(numstates)
print(distance)

24576
[1, 2, 2, 3, 3, 4]


In [4]:
#get probs
actionprobs=[]
if(actionDist=="uniform"):
    for i in range(numAction):
        actionprobs.append(float(1)/numAction)
        
elif(actionDist=="fromTraining"):
    counts=train.groupby(["action","color"]).size()
    totalnum=len(train["action"])
    for action in ['store','restore']:
        for color in ['red','white','blue']:
            actionprobs.append(float(counts[action][color])/totalnum)
    
    
actionprobs

[0.24686157912124215,
 0.1278493557978196,
 0.12528906508093823,
 0.24686157912124215,
 0.1278493557978196,
 0.12528906508093823]

In [5]:
#helper functions to switch an integer to a state and back
def intToState(statenum):
    action=int(statenum/(fieldstates**fields))
    tmp=statenum%(fieldstates**fields)
    retstate=[]
    for i in range(1,fields+1):
        fieldi=int(tmp/(fieldstates**(fields-i)))
        tmp=statenum%(fieldstates**(fields-i))
        retstate.append(fieldi)
        
    retstate.append(action)
    return retstate
    
def stateToInt(state):
    retval=0
    for i in range(0,len(state)):
        retval+=state[i]*fieldstates**i
    return retval
    
print(intToState(1))
print(intToState(2))
print(intToState(3))
print(intToState(4))
print(intToState(12287))
print(stateToInt((3,3,3,3,3,3,2)))

[0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 2, 0]
[0, 0, 0, 0, 0, 3, 0]
[0, 0, 0, 0, 1, 0, 0]
[3, 3, 3, 3, 3, 3, 2]
12287


In [6]:
#helper function to normalize probability vectors to one
def normalize(problist):
    sums=sum(problist)
    factor=float(1)/sums
    for i in range (0,len(problist)):
        problist[i]=factor*problist[i]
    return problist

normalize([1.0,0.1,0.2,0.3])

[0.625, 0.0625, 0.125, 0.1875]

In [26]:
#helper functions for the population of transition prob matrices

#returns true iff specific target for is valid in state
def isValidTarget(action,statenum,targetslot):
    state=intToState(statenum)
    
    stateaction=state[-1]
    
    #forbid actions that are not enforced by the state
    if(action!=stateaction):
        return False
    
    #check if enforced action makes any sense
    
    #3,4,5 are restore
    if(int(action/3)==1):
        target=action%3
        if(target==state[targetslot]):
            return True
        else:
            return False
    
    #0,1,2 are store
    elif(int(action/3)==0):
        #3 is empty
        if(state[targetslot]==3):
            return True
        else:
            return False
    else:
        print("wtf!")
        return False


#returns true iff action is possible in state
def isValid(action,statenum):
    state=intToState(statenum)
    
    stateaction=state[-1]
    
    #forbid actions that are not enforced by the state
    if(action!=stateaction):
        return False
    
    #check if enforced action makes any sense
    
    #3,4,5 are restore
    if(int(action/3)==1):
        target=action%3
        if(target in state[0:-1]):
            return True
        else:
            return False
    
    #0,1,2 are store
    elif(int(action/3)==0):
        #3 is empty
        if(3 in state[0:-1]):
            return True
        else:
            return False
    else:
        print("wtf!")
        return False

#returns list of tuples of followstate and probability
def getRejectionFollowStates(statenum):
    retlist=[]
    retstates=[]
    retprobs=[]
    
    state=intToState(statenum)
    
    for action in range(0,numAction):
        newstate=state.copy()
        newstate[-1]=action
        retstates.append(stateToInt(newstate))
        retprobs.append(actionprobs[action])
    
    for i in range(0,len(retstates)):
        retlist.append((retstates[i],retprobs[i]))
    
    return retlist
    
    
    
#returns a list of tuples of followstates as integer , their normalized probabilities, and their rewards
#it is assumed that there are valid transitions for the given state and targetslot
def getValidFollowStates(statenum,targetslot):
    state=intToState(statenum)
    action=state[-1]
    
    #3,4,5 are restore
    if(int(action/3)==1):
        targetcolor=action%3
        
        #get targetstates by combining followup warehouse layouts with followup actions
        targetstates=[]
        targetprobs=[]
        targetrewards=[]
        
        for fua in range(0,numAction):
            newstate=state.copy()
            newstate[-1]=fua
            newstate[targetslot]=3
            #get prob as product of actionprob and locationprob
            newstateprob=actionprobs[fua]
            
            targetstates.append(stateToInt(newstate))
            targetprobs.append(newstateprob)
            #doing a valid move is rewarded, less with more distance traveled
            targetrewards.append(2*max(distance)-2*distance[targetslot]+1)
        
        retlist=[]
        normalize(targetprobs)
        for i in range(0,len(targetstates)):
            retlist.append((targetstates[i],targetprobs[i],targetrewards[i]))
        return retlist
                    
    #0,1,2 are store
    elif(int(action/3)==0):
        storeitem=action%3

        #get targetstates by combining followup warehouse layouts with followup actions
        targetstates=[]
        targetprobs=[]
        targetrewards=[]
        
       
        for fua in range(0,numAction):
            newstate=state.copy()
            newstate[-1]=fua
            newstate[targetslot]=storeitem
            #get prob as product of actionprob and locationprob
            newstateprob=actionprobs[fua]
            targetstates.append(stateToInt(newstate))
            targetprobs.append(newstateprob)
            #doing a valid move is rewarded, less with more distance traveled
            targetrewards.append(2*max(distance)-2*distance[targetslot])
                
        retlist=[]
        normalize(targetprobs)
        
        for i in range(0,len(targetstates)):
            retlist.append((targetstates[i],targetprobs[i],targetrewards[i]))
        return retlist
        
        
    else:
        print("wtf state!")
        return None
    
    
    
print(isValid(0,165))
print(isValid(2,12287))
print()
print(intToState(165))
for item in getRejectionFollowStates(165):
    print(item)
print()
print(intToState(12287))
for item in getValidFollowStates(12287,0):
    print(str(item[1])+","+str(item[0])+","+str(item[2]))
print()
for item in getValidFollowStates(12287,1):
    print(str(item[1])+","+str(item[0])+","+str(item[2]))
print()
for item in getValidFollowStates(12287,2):
    print(str(item[1])+","+str(item[0])+","+str(item[2]))
print()
for item in getValidFollowStates(12287,3):
    print(str(item[1])+","+str(item[0])+","+str(item[2]))
print()
for item in getValidFollowStates(12287,4):
    print(str(item[1])+","+str(item[0])+","+str(item[2]))
print()
for item in getValidFollowStates(12287,5):
    print(str(item[1])+","+str(item[0])+","+str(item[2]))

False
True

[0, 0, 2, 2, 1, 1, 0]
(1440, 0.24686157912124215)
(5536, 0.1278493557978196)
(9632, 0.12528906508093823)
(13728, 0.24686157912124215)
(17824, 0.1278493557978196)
(21920, 0.12528906508093823)

[3, 3, 3, 3, 3, 3, 2]
0.24686157912124215,4094,6
0.1278493557978196,8190,6
0.12528906508093823,12286,6
0.24686157912124215,16382,6
0.1278493557978196,20478,6
0.12528906508093823,24574,6

0.24686157912124215,4091,4
0.1278493557978196,8187,4
0.12528906508093823,12283,4
0.24686157912124215,16379,4
0.1278493557978196,20475,4
0.12528906508093823,24571,4

0.24686157912124215,4079,4
0.1278493557978196,8175,4
0.12528906508093823,12271,4
0.24686157912124215,16367,4
0.1278493557978196,20463,4
0.12528906508093823,24559,4

0.24686157912124215,4031,2
0.1278493557978196,8127,2
0.12528906508093823,12223,2
0.24686157912124215,16319,2
0.1278493557978196,20415,2
0.12528906508093823,24511,2

0.24686157912124215,3839,2
0.1278493557978196,7935,2
0.12528906508093823,12031,2
0.24686157912124215,16127,2
0.127

In [8]:
#create list of transition probability matrices
transitions=[]
rewards=[]
#one matrix for each action, an action is the selection of a specific field as target for the order
for targetslot in range(0,fields):
    #float 32 to limit memory usage (lowest for scipy.sparse)
    #and because the probs from training are not too terribly precise anyways so loss should be small
    workmat=np.zeros((numstates,numstates), dtype=np.float32)
    rewardmat=np.zeros((numstates,numstates), dtype=np.float32)
    
    for fromidx in range(numstates):
        fromstate=intToState(fromidx)
        actidx=fromstate[-1]
        #skip if action is invalid/not possible 
        if(not isValid(actidx,fromidx)):
            #act as if the current action was rejected and the next one is drawn
            fus=getRejectionFollowStates(fromidx)
            for nxtstate in fus:
                #set probability according to occurence rate of new action
                workmat[fromidx][nxtstate[0]]=nxtstate[1]
                #penalize this HEAVILY as it is equivalent to a storage system failure
                rewardmat[fromidx][nxtstate[0]]=-100.0
            continue
        
        #do the same if action is valid but target is not
        if(not isValidTarget(actidx,fromidx,targetslot)):
            #act as if the current action was rejected and the next one is drawn
            fus=getRejectionFollowStates(fromidx)
            for nxtstate in fus:
                #set probability according to occurence rate of new action
                workmat[fromidx][nxtstate[0]]=nxtstate[1]
                #penalize this HEAVILY as it is equivalent to a storage system failure
                rewardmat[fromidx][nxtstate[0]]=-100.0
            continue
        
        #in any other case get list of possible follow up states
        fus=getValidFollowStates(fromidx,targetslot)
        for nxtstate in fus:
            workmat[fromidx][nxtstate[0]]=nxtstate[1]
            rewardmat[fromidx][nxtstate[0]]=nxtstate[2]
          
    workmat=csr_matrix(workmat, dtype=np.float32)
    rewardmat=csr_matrix(rewardmat)
    transitions.append(workmat)
    rewards.append(rewardmat)       
    
tmatrices=transitions
#tmatrices=np.array(transitions)
#tmatrices=sparse.csr_matrix(transitions, dtype=np.float16)
print(tmatrices)
rmatrices=rewards
#rmatrices=np.array(rewards)
#rmatrices=sparse.csr_matrix(rewards, dtype=np.float16)
print(rmatrices)

[<24576x24576 sparse matrix of type '<class 'numpy.float32'>'
	with 147456 stored elements in Compressed Sparse Row format>, <24576x24576 sparse matrix of type '<class 'numpy.float32'>'
	with 147456 stored elements in Compressed Sparse Row format>, <24576x24576 sparse matrix of type '<class 'numpy.float32'>'
	with 147456 stored elements in Compressed Sparse Row format>, <24576x24576 sparse matrix of type '<class 'numpy.float32'>'
	with 147456 stored elements in Compressed Sparse Row format>, <24576x24576 sparse matrix of type '<class 'numpy.float32'>'
	with 147456 stored elements in Compressed Sparse Row format>, <24576x24576 sparse matrix of type '<class 'numpy.float32'>'
	with 147456 stored elements in Compressed Sparse Row format>]
[<24576x24576 sparse matrix of type '<class 'numpy.float32'>'
	with 147456 stored elements in Compressed Sparse Row format>, <24576x24576 sparse matrix of type '<class 'numpy.float32'>'
	with 147456 stored elements in Compressed Sparse Row format>, <24576

In [9]:
#check weather created matrices are valid for mdp toolbox
#for matrix in tmatrices:
#    print(mdptoolbox.util.isSquare(matrix))
#    print(mdptoolbox.util.isStochastic(matrix))
    

#for matrix in rmatrices:
#    print(mdptoolbox.util.isSquare(matrix))
    
mdptoolbox.util.check(tmatrices, rmatrices)

#create several mdp algorithms to solve it
#finite Horizon
fh = mdptoolbox.mdp.FiniteHorizon(tmatrices, rmatrices, 0.9 , 3 )

#policy iteration
pi = mdptoolbox.mdp.PolicyIteration(tmatrices, rmatrices, 0.9, max_iter=10)

#qlearning
ql = mdptoolbox.mdp.PolicyIterationModified(tmatrices, rmatrices, 0.9)

#value iteration
vi = mdptoolbox.mdp.ValueIteration(tmatrices, rmatrices, discount=0.9, epsilon=0.01)


In [10]:
#run the mdps to compute policies
#finiteHorizon
fh.run()
fhpolicy=fh.policy
fhpolicy

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [5, 5, 5],
       [0, 0, 0]])

In [11]:
#policy iteration
pi.run()
pipolicy=pi.policy
pipolicy

(0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 4,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 4,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,


In [12]:
#qlearning
ql.run()
qlpolicy=ql.policy
qlpolicy

(0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 4,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,


In [13]:
#value iteration
vi.run()
vipolicy=vi.policy
vipolicy

(0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 4,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 5,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,


In [30]:
#create greedy and random as comparison algorithms
#each algorithm takes in a state as input and puts out the resulting state as output
#each algorithm will prioritize to not reject the action over rejecting it if possible

def greedy(instate):
    state=intToState(instate)
    action=state[-1]
    #rejection was necessary
    if(not isValid(action,instate)):
        #followupstate action does not matter, just return instate again
        return instate
    #action was executed, get target states
    targets=[]
    #store
    if(int(action/3)==0):
        
        for i in range(0,fields):
            if(state[i]==3):
                targets.append(i)
    #retrieve
    elif(int(action/3)==1):
       
        targetblock=action%3
        for i in range(0,fields):
            if(state[i]==targetblock):
                targets.append(i)
    
    #select target greedily
    target=min(targets)
    
    #execute move without changing action
    newstate=state.copy()
    #store
    if(int(action/3)==0):
        newstate[target]=action%3
    #retrieve
    elif(int(action/3)==1):
        newstate[target]=3
        
    return stateToInt(newstate)
        
def random(instate):
    state=intToState(instate)
    action=state[-1]
    #rejection was necessary
    if(not isValid(action,instate)):
        #followupstate action does not matter, just return instate again
        return instate
    #action was executed, get target states
    targets=[]
    #store
    if(int(action/3)==0):
        for i in range(0,fields):
            if(state[i]==3):
                targets.append(i)
    #retrieve
    elif(int(action/3)==1):
        targetblock=action%3
        for i in range(0,fields):
            if(state[i]==targetblock):
                targets.append(i)
    
    #select target randomly
    target=rand.choice(targets)
    
    #execute move without changing action
    newstate=state.copy()
    #store
    if(int(action/3)==0):
        newstate[target]=action%3
    #retrieve
    elif(int(action/3)==1):
        newstate[target]=3
        
    return stateToInt(newstate)

print(greedy(12287))
print(random(12287))
print(random(12287))
print(random(12287))

12286
12286
12031
11263


In [31]:
#helper function that executes a policy on a given state, returns the integer representation of resulting state
def executePolicy(instate,policy):
    
    state=intToState(instate)
    action=state[-1]
    if(not isValid(action,instate)):
        #followupstate action does not matter, just return instate again
        return instate
    
    target=policy[instate]
    
    #execute move without changing action
    newstate=state.copy()
    #store
    if(int(action/3)==0):
        newstate[target]=action%3
    #retrieve
    elif(int(action/3)==1):
        newstate[target]=3
        
    return stateToInt(newstate)



In [32]:
#define a magic value for rejection
magic=0xbadac1

#helper function that gives the cost of a given state transition in the form of distance traveled
def getTransitionCost(fromState,toState):
    state1=intToState(fromState)
    state2=intToState(toState)
    
    change=-1
    #check which field changed
    for i in range(0,fields):
        if(not state1[i]==state2[i]):
            change=i
    #if no state changed, there was a rejection
    if(change==-1):
        return magic
    #else return distance traveled as twice the distance to the changed space
    else:
        return 2*distance[fields-change-1]
    
print(hex(getTransitionCost(165,933)))
print(str(intToState(12287))+"  "+str(intToState(4094)))
print(getTransitionCost(12287,4094))
print(str(intToState(12287))+"  "+str(intToState(4091)))
print(getTransitionCost(12287,4091))
print(str(intToState(12287))+"  "+str(intToState(4079)))
print(getTransitionCost(12287,4079))
print(str(intToState(12287))+"  "+str(intToState(4031)))
print(getTransitionCost(12287,4031))
print(str(intToState(12287))+"  "+str(intToState(3839)))
print(getTransitionCost(12287,3839))
print(str(intToState(12287))+"  "+str(intToState(3071)))
print(getTransitionCost(12287,3071))


0x6
[3, 3, 3, 3, 3, 3, 2]  [3, 3, 3, 3, 3, 2, 0]
2
[3, 3, 3, 3, 3, 3, 2]  [3, 3, 3, 3, 2, 3, 0]
4
[3, 3, 3, 3, 3, 3, 2]  [3, 3, 3, 2, 3, 3, 0]
4
[3, 3, 3, 3, 3, 3, 2]  [3, 3, 2, 3, 3, 3, 0]
6
[3, 3, 3, 3, 3, 3, 2]  [3, 2, 3, 3, 3, 3, 0]
6
[3, 3, 3, 3, 3, 3, 2]  [2, 3, 3, 3, 3, 3, 0]
8


In [33]:
#helper functions for test execution
#creates integer representation of action shown in line
def parseLine(line):
    action=0
    line=line.strip('\n')
    if(line.split('\t')[0]=="store"):
        action+=0
    elif(line.split('\t')[0]=="restore"):
        action+=3
    else:
        print("something went wrong")
        return -1
    if(line.split('\t')[1]=="red"):
        action+=0
    elif(line.split('\t')[1]=="white"):
        action+=1
    elif(line.split('\t')[1]=="blue"):
        action+=2
    else:
        print("something went wrong")
        return -1
    
    
    return action

print(parseLine("eat\tred"))
print(parseLine("store\tgreen"))
print(parseLine("store\tred"))
print(parseLine("restore\twhite"))
print(parseLine("restore\tblue"))

something went wrong
-1
something went wrong
-1
0
4
5


In [34]:
#helper function to simulate the execution of the test procedure with the given strategy
def executeTest(strategy):
    print("test: "+strategy+":")
    #initialState is an empty warehouse
    laststate=[3,3,3,3,42]
    
    traveledDistance=0
    rejections=0
    #reset test file to start
    test.seek(0)
    for line in test:
        action=parseLine(line)
        #the execution order formulated as a state:
        orderstate=laststate.copy()
        orderstate[-1]=action
        orderstatei=stateToInt(orderstate)
        
        #print(orderstate)
        #get the next state based on execution strategy
        nextstate=0
        if(strategy=="random"):
            nextstate=random(orderstatei)
        elif(strategy=="greedy"):
            nextstate=greedy(orderstatei)
        elif(strategy=="finiteHorizon"):
            nextstate=executePolicy(orderstatei,fhpolicy)
        elif(strategy=="policyIteration"):
            nextstate=executePolicy(orderstatei,pipolicy)
        elif(strategy=="qlearn"):
            nextstate=executePolicy(orderstatei,qlpolicy)
        elif(strategy=="valueIteration"):
            nextstate=executePolicy(orderstatei,vipolicy)
        else:
            print("strategy not supported: "+strategy)
            return
        
        costs=getTransitionCost(orderstatei,nextstate)
        #print(orderstate)
        if(costs==magic):
            #print("rejected "+line.strip('\n'))
            #if(rejections==2):
            #    return
            rejections+=1
        else:
            traveledDistance+=costs
            
        laststate=intToState(nextstate)
            
    print("traveled "+str(traveledDistance)+" units of distance")
    print("rejected "+str(rejections)+" orders")

    
#executeTest("Schwingschleifer")
executeTest("greedy")
executeTest("random")
executeTest("policyIteration")
executeTest("qlearn")
executeTest("valueIteration")

test: greedy:
traveled 130 units of distance
rejected 5 orders
test: random:
traveled 136 units of distance
rejected 5 orders
test: policyIteration:
traveled 130 units of distance
rejected 5 orders
test: qlearn:
traveled 130 units of distance
rejected 5 orders
test: valueIteration:
traveled 130 units of distance
rejected 5 orders


In [35]:
train=open("training.txt","r")

#helper function to simulate the execution of the train procedure with the given strategy
def executeTestOnTrain(strategy):
    print("train: "+strategy+":")
    #initialState is an empty warehouse
    laststate=[3,3,3,3,42]
    
    traveledDistance=0
    rejections=0
    #reset test file to start
    train.seek(0)
    for line in train:
        action=parseLine(line)
        #the execution order formulated as a state:
        orderstate=laststate.copy()
        orderstate[4]=action
        orderstatei=stateToInt(orderstate)
        
        #get the next state based on execution strategy
        nextstate=0
        if(strategy=="random"):
            nextstate=random(orderstatei)
        elif(strategy=="greedy"):
            nextstate=greedy(orderstatei)
        elif(strategy=="finiteHorizon"):
            nextstate=executePolicy(orderstatei,fhpolicy)
        elif(strategy=="policyIteration"):
            nextstate=executePolicy(orderstatei,pipolicy)
        elif(strategy=="qlearn"):
            nextstate=executePolicy(orderstatei,qlpolicy)
        elif(strategy=="valueIteration"):
            nextstate=executePolicy(orderstatei,vipolicy)
        else:
            print("strategy not supported: "+strategy)
            return
        
        costs=getTransitionCost(orderstatei,nextstate)
        #print(orderstate)
        if(costs==magic):
            #print("rejected "+line.strip('\n'))
            #if(rejections==2):
            #    return
            rejections+=1
        else:
            traveledDistance+=costs
            
        laststate=intToState(nextstate)
            
    print("traveled "+str(traveledDistance)+" units of distance")
    print("rejected "+str(rejections)+" orders")

executeTestOnTrain("greedy")
executeTestOnTrain("random")
executeTestOnTrain("policyIteration")
executeTestOnTrain("qlearn")
executeTestOnTrain("valueIteration")

train: greedy:
traveled 11498 units of distance
rejected 8094 orders
train: random:
traveled 11498 units of distance
rejected 8094 orders
train: policyIteration:
traveled 13588 units of distance
rejected 7665 orders
train: qlearn:
traveled 13588 units of distance
rejected 7665 orders
train: valueIteration:
traveled 13588 units of distance
rejected 7665 orders


In [36]:
#eveluate different values for discount
for discount in [0.5,0.9,0.95]:
    print("DISCOUNT: "+str(discount))
    #define
    #policy iteration
    pi = mdptoolbox.mdp.PolicyIteration(tmatrices, rmatrices, discount,max_iter=10)
    #qlearning
    ql = mdptoolbox.mdp.PolicyIterationModified(tmatrices, rmatrices, discount)
    #value iteration
    vi = mdptoolbox.mdp.ValueIteration(tmatrices, rmatrices, discount)
    #train
    start=time.time()
    pi.run()
    pipolicy=pi.policy
    end=time.time()
    t=end-start
    print("Policy iteration training time: "+str(t))
    start=time.time()
    ql.run()
    qlpolicy=ql.policy
    end=time.time()
    t=end-start
    print("Q learn training time: "+str(t))
    start=time.time()
    vi.run()
    vipolicy=vi.policy
    end=time.time()
    t=end-start
    print("Value iteration training time: "+str(t))
    #run
    executeTest("policyIteration")
    executeTest("qlearn")
    executeTest("valueIteration")
    executeTestOnTrain("policyIteration")
    executeTestOnTrain("qlearn")
    executeTestOnTrain("valueIteration")
    
    
executeTest("greedy")
executeTest("random")
executeTestOnTrain("greedy")
executeTestOnTrain("random")

DISCOUNT: 0.5
Policy iteration training time: 763.5396375656128
Q learn training time: 7.562990665435791
Value iteration training time: 0.07106494903564453
test: policyIteration:
traveled 130 units of distance
rejected 5 orders
test: qlearn:
traveled 130 units of distance
rejected 5 orders
test: valueIteration:
traveled 130 units of distance
rejected 5 orders
train: policyIteration:
traveled 11498 units of distance
rejected 8094 orders
train: qlearn:
traveled 11498 units of distance
rejected 8094 orders
train: valueIteration:
traveled 11498 units of distance
rejected 8094 orders
DISCOUNT: 0.9
Policy iteration training time: 744.8110594749451
Q learn training time: 28.531759023666382
Value iteration training time: 0.3667168617248535
test: policyIteration:
traveled 130 units of distance
rejected 5 orders
test: qlearn:
traveled 130 units of distance
rejected 5 orders
test: valueIteration:
traveled 130 units of distance
rejected 5 orders
train: policyIteration:
traveled 13588 units of dist